<a href="https://colab.research.google.com/github/dominiksakic/bagOfWords/blob/main/bagOfWords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Download and unzip data
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  6210k      0  0:00:13  0:00:13 --:--:--  9.7M


In [19]:
import os, pathlib, shutil, random

# Create a Validation set
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val' # No need for string concat, thanks to pathlib
train_dir = base_dir / 'train'
for category in ("neg", "pos"):
  os.makedirs(val_dir / category, exist_ok=True)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 *len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category /fname, val_dir / category / fname)

In [20]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
